In [7]:
import torch
import torch.nn as nn
import numpy as np

class MLP(nn.Module):
    """
    a simple MLP with two hidden layers
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [10]:
class CustomDataLoader:
    """
    A custom data loader that subsamples the mini-batch based on high error. The MSELoss is used as 
    the loss function since this is a regression problem. The error is computed for each example in
    the mini-batch using the loss, the examples are sorted based on the error, and a subset of 
    examples with high error is selected. The subsample_fraction hyperparameter determines the 
    fraction of examples to keep.
    """
    def __init__(self, dataset, batch_size, subsample_fraction):
                """
        Args:
        - dataset: a PyTorch Dataset object
        - batch_size: an integer indicating the batch size
        - subsample_fraction: a float between 0 and 1 indicating the fraction of examples to use for 
            backpropagation
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.subsample_fraction = subsample_fraction
    
    def __iter__(self):
        """
        Returns an iterator that generates subsampled minibatches from the dataset.
        """
        indices = torch.randperm(len(self.dataset))
        for start in range(0, len(self.dataset), self.batch_size):
            end = min(start + self.batch_size, len(self.dataset))
            inputs, targets = self.dataset[indices[start:end]]
            
            with torch.no_grad():
                # Forward pass to obtain outputs and compute loss
                outputs = model(inputs)
                loss = nn.MSELoss()(outputs, targets)
                
                # Compute errors and subsample indices based on highest errors
                errors = loss.detach().cpu().numpy()
                    # detach() returns a new Tensor, detached from the current graph.
                    # cpu() returns a copy of this object in CPU memory.
                    # numpy() converts a tensor object into an numpy.ndarray object
                sorted_idx = np.argsort(errors)[::-1] 
                    # np.argsort() returns the indices that would sort an array. 
                    # np.argsort([3, 1, 4, 2]) would return [1, 3, 0, 2]
                    # [::-1] changes order of indices to descending order of error.
                    # the result is the array of indices that correspond to the examples with the 
                    # highest errors in the current minibatch
                subset_idx = sorted_idx[:int(len(sorted_idx) * self.subsample_fraction)]
                    # selects a subset of these indices based on the subsample_fraction parameter,
                    # and the resulting subset_idx array is used to obtain a subset of the inputs 
                    # and targets for backpropagation.
                    
            # Yield subsampled inputs and targets
            yield inputs[subset_idx].clone(), targets[subset_idx].clone()

A randomly generated dataset is used for demonstration purposes, where each example has 10 features and 1 target value. We're using the custom data loader with a batch size of 32 and a subsample fraction of 0.5. The optimizer used is stochastic gradient descent with a learning rate of 0.1. We train the model for 10 epochs.

In [11]:
# Set up the model, data, and optimizer
model = MLP(input_size=10, hidden_size=64, output_size=1)
train_dataset = torch.utils.data.TensorDataset(torch.randn(1000, 10), torch.randn(1000, 1))
train_loader = CustomDataLoader(train_dataset, batch_size=32, subsample_fraction=0.5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Train the model
for epoch in range(10):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.MSELoss()(outputs, targets)
        loss.backward()
        optimizer.step()

ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 